In [2]:
from django.core.management import setup_environ
import settings
setup_environ(settings)

'/home/wahern/projects/millstone/genome_designer'

In [3]:
import os
from main.models import ExperimentSampleToAlignment
sample_alignment = ExperimentSampleToAlignment.objects.get(experiment_sample__label='lib1_rec07')

genome_finish_dir = os.path.join(sample_alignment.get_model_data_dir(), 'genome_finishing')

In [6]:
genome_finish_dir

'/home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/6e766eb8/alignment_groups/9c54183d/sample_alignments/ca0c79af/genome_finishing'

In [7]:
read_unpacking_dir = os.path.join(genome_finish_dir, '0', 'read_unpacking')
#os.mkdir(read_unpacking_dir)

In [8]:
import subprocess
cmd = 'ls'

fn = os.path.join(read_unpacking_dir, 'test_output.txt')
with open(fn, 'w') as fh:
    subprocess.call(cmd, shell=True, stdout=fh)

In [15]:
velvet_dir = os.path.join(settings.TOOLS_DIR, 'velvet')
extract_contig_reads_executable = os.path.join(velvet_dir, 'contrib/extractContigReads/extractContigReads.pl')

assembly_dir = os.path.join(genome_finish_dir, '0', 'velvet_k21')

cmd = [extract_contig_reads_executable, '1', assembly_dir]
cmd = ' '.join(cmd)
print cmd

fn = os.path.join(read_unpacking_dir, 'extracted.fa')
with open(fn, 'w') as fh:
    subprocess.call(cmd, shell=True, stdout=fh)

/home/wahern/projects/millstone/genome_designer/conf/../tools/velvet/contrib/extractContigReads/extractContigReads.pl 1 /home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/6e766eb8/alignment_groups/9c54183d/sample_alignments/ca0c79af/genome_finishing/0/velvet_k21


In [62]:
import pysam
import re
p1 = re.compile('>(\S+)/(\d)')

# reads = []
# with open(fn) as fh:
#     counter = 0
#     for line in fh:
#         #print 'line', counter, ':'
#         print line
        
#         m1 = p1.match(line)
#         if m1:
#             reads.append({'read_id': m1.group(1), 'read_number': m1.group(2)})
#             print 're match:\n\tgroup(1):', m1.group(1), '\n\tgroup(2):', m1.group(2)
#         counter += 1
#         if counter == 4:
#             break

# print reads

sv_indicants_path = os.path.join(genome_finish_dir, '0/bwa_align.SV_indicants_with_pairs.bam')
sam_file = pysam.AlignmentFile(sv_indicants_path)
counter = 0
for read in sam_file:
    print 'read.query_name:', read.query_name
    counter += 1
    if counter == 5:
        break

read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:71563
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:71563
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:95656
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1126:95656
read.query_name: HSQ700642:127:D0F9HACXX:5:1101:1127:6749


In [18]:
from main.models import Dataset
fq1_path = sample_alignment.experiment_sample.dataset_set.get(type=Dataset.TYPE.FASTQ1).wrap_if_compressed()
print fq1_path

<(gzip -dc /home/wahern/projects/millstone/genome_designer/conf/../temp_data/projects/6e766eb8/samples/eab3affa/tmprbj2gz_recoli_lib1_rec07_1.fq.gz)


In [10]:
import os
import re
import pysam
from collections import defaultdict

velvet_dir = os.path.join(settings.TOOLS_DIR, 'velvet')
extract_contig_reads_executable = os.path.join(velvet_dir, 'contrib/extractContigReads/extractContigReads.pl')

sample_alignment = ExperimentSampleToAlignment.objects.get(experiment_sample__label='ins_1kb_sample')
genome_finish_dir = os.path.join(sample_alignment.get_model_data_dir(), 'genome_finishing')
# print 'genome_finish_dir:', genome_finish_dir
assembly_dir = os.path.join(genome_finish_dir, '1', 'velvet_k21')

read_unpacking_dir = os.path.join(genome_finish_dir, '1', 'read_unpacking')
if not os.path.exists(read_unpacking_dir):
    os.mkdir(read_unpacking_dir)

contig_number = 1
cmd = [extract_contig_reads_executable, str(contig_number), assembly_dir]
cmd = ' '.join(cmd)

contig_reads_fasta = os.path.join(read_unpacking_dir, 'contig_' + str(contig_number) + '_reads.fa')
if not os.path.exists(contig_reads_fasta):
    with open(contig_reads_fasta, 'w') as fh:
        subprocess.call(cmd, shell=True, stdout=fh)

p1 = re.compile('>(\S+)/(\d)')
contig_reads = defaultdict(list)
with open(contig_reads_fasta) as fh:
    for line in fh:
        m1 = p1.match(line)
        if m1:
            read_id = m1.group(1)
            read_number = int(m1.group(2))
            contig_reads[read_id].append(read_number)

sv_indicants_path = os.path.join(genome_finish_dir, '1/bwa_align.SV_indicants_with_pairs.bam')
sam_file = pysam.AlignmentFile(sv_indicants_path)
sv_indicant_reads_in_contig = []
# counter = 0
for read in sam_file:
    if read.is_read1:
        read_number = 1
    elif read.is_read2:
        read_number = 2
    else:
        raise Exception('Read is neither read1 nor read2')

    contig_read_numbers = contig_reads.get(read.query_name, [])
    if read_number in contig_read_numbers:
        sv_indicant_reads_in_contig.append(read)

# Verification
number_contig_reads = 0
for cr in contig_reads.values():
    number_contig_reads += len(cr)

print 'number_contig_reads:', number_contig_reads
print 'len(sv_indicant_reads_in_contig):', len(sv_indicant_reads_in_contig)
assert number_contig_reads == len(sv_indicant_reads_in_contig)

number_contig_reads: 588
len(sv_indicant_reads_in_contig): 588


In [17]:
all_ref_pos = []
for read in sv_indicant_reads_in_contig:
    all_ref_pos.extend(read.get_reference_positions())

In [23]:
right_clip_positions = []
left_clip_positions = []


BAM_CSOFT_CLIP = 4
    BAM_CHARD_CLIP = 5
    clip_codes = [BAM_CSOFT_CLIP, BAM_CHARD_CLIP]
for read in sv_indicant_reads_in_contig:
    
    

20494